<a href="https://colab.research.google.com/github/thecrazyphysicist369/Forensic-Project/blob/main/code/Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null
!pip install transformers
!pip install emoji

     |████████████████████████████████| 2.0MB 7.7MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 3.2MB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0435db970a2ed7f3a7c88c7dfa4300e951faa2fed2850cf4873d8ac3772f13b0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 133kB 7.1MB/s 


In [2]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import emoji
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

In [24]:
featured_data = pd.read_csv('/content/featured_dataset.csv', dtype={"A" :"string",
                                                           "B" :"string",
                                                           "C" :"string",
                                                           "D" :"string",
                                                           "E" :"string",
                                                           "F" :"string",
                                                           "G" :"string",
                                                           "H" :"string",
                                                           "I" :"string",
                                                           "J" :"string",
                                                           "K" :"string",
                                                           "L" :"string",
                                                           "M" :"string",
                                                           "Y" :"string",
                                                           })

In [56]:
featured_data.drop(columns='Y')


,A,B,C,D,E,F,G,H,I,J,K,L,M
0,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,poll([{fd=4,events=POLLIN},{fd=5,events=POLLIN},{fd=7,events=POLLIN},{fd=28,events=POLLIN},{fd=41,events=POLLIN}],5,0) = 0 (Timeout),<NA>
2,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,poll([{fd=4,events=POLLIN},{fd=5,events=POLLIN},{fd=7,events=POLLIN},{fd=28,events=POLLIN},{fd=41,events=POLLIN}],5,0) = 0 (Timeout),<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3654,poll([{fd=4,events=POLLIN},{fd=5,events=POLLIN},{fd=7,events=POLLIN},{fd=28,events=POLLIN},{fd=41,events=POLLIN}],5,0) = 0 (Timeout),<NA>
3655,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3656,recvmsg(4,{msg_namelen=0},0) = -1 EAGAIN (Resource temporarily...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [18]:
type(data["Y"][0])

str

In [27]:
id = {"1" : 1,
      "0" : 0}
featured_data['Y'] = featured_data['Y'].map(id) 

# Encoding Data

In [61]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(featured_data.Y)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)
featured_data = featured_data.drop(columns='Y')

In [38]:
len(x_test)

732

In [62]:
x_train, x_test, y_train, y_test = train_test_split(featured_data,Y, random_state=1995, test_size=0.2, shuffle=False)

In [58]:
len(x_train)

2926

In [71]:
y_train.shape

(2926, 3)

#LSTM


##Training

In [65]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 160
Epoch = 5
token.fit_on_texts(list(x_train) + list(x_test))
#x_train_pad = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=max_len)
#x_test_pad = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=max_len)

In [69]:
x_train_pad.shape

(13, 160)

In [70]:
w_idx = token.word_index
len(w_idx)

13

In [47]:
embed_dim = 160
lstm_out = 250

model = Sequential()
model.add(Embedding(x_train.shape[1]))
#model.add(Embedding(len(w_idx) +1 , embed_dim,input_length = x_test_pad.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.core.Dense(2, activation='softmax'))
#adam rmsprop 
model.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 160)          2400      
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 160, 160)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 250)               411000    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 502       
Total params: 413,902
Trainable params: 413,902
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
batch_size = 32

In [68]:
model.fit(x_train_pad, y_train, epochs = Epoch, batch_size=batch_size,validation_data=(x_test_pad, y_test))

ValueError: ignored

In [ ]:
def get_sentiment(model,text):
    text = clean_text(text)
    #tokenize
    twt = token.texts_to_sequences([text])
    twt = sequence.pad_sequences(twt, maxlen=max_len, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [ ]:
def plot_result(df):
    #colors=['#D50000','#000000','#008EF8','#F5B27B','#EDECEC','#D84A09','#019BBD','#FFD000','#7800A0','#098F45','#807C7C','#85DDE9','#F55E10']
    #fig = go.Figure(data=[go.Pie(labels=df.sentiment,values=df.percentage, hole=.3,textinfo='percent',hoverinfo='percent+label',marker=dict(colors=colors, line=dict(color='#000000', width=2)))])
    #fig.show()
    colors={'love':'rgb(213,0,0)','empty':'rgb(0,0,0)',
                    'sadness':'rgb(0,142,248)','enthusiasm':'rgb(245,178,123)',
                    'neutral':'rgb(237,236,236)','worry':'rgb(216,74,9)',
                    'surprise':'rgb(1,155,189)','fun':'rgb(255,208,0)',
                    'hate':'rgb(120,0,160)','happiness':'rgb(9,143,69)',
                    'boredom':'rgb(128,124,124)','relief':'rgb(133,221,233)',
                    'anger':'rgb(245,94,16)'}
    col_2={}
    for i in result.sentiment.to_list():
        col_2[i]=colors[i]
    fig = px.pie(df, values='percentage', names='sentiment',color='sentiment',color_discrete_map=col_2,hole=0.3)
    fig.show()

##Testing

In [ ]:
result =get_sentiment(model,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
plot_result(result)
result =get_sentiment(model,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
plot_result(result)
result =get_sentiment(model,"I hate this game so much,It make me angry all the time ")
plot_result(result)